In [1]:
import cv2
import os
import numpy as np
import pickle

In [2]:
# Image Enhancing
def preprocess_img(img):
    # gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # enhance contrast
    clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(16,16))
    equalized_image = clahe.apply(gray)

    # Gaussian blur
    blurred_image = cv2.GaussianBlur(equalized_image, (5, 5), 0)

    # Canny edge
    edges = cv2.Canny(blurred_image, threshold1=60, threshold2=110)
    return edges

target_img = cv2.imread('./wortel.png')

final_target = preprocess_img(target_img)

# cv2.imshow('final',final_target)
# cv2.waitKey(0)



In [3]:
# Feature Detection

sift = cv2.SIFT_create()
orb = cv2.ORB_create()

def detect_features(img):
    # SIFT
    kp_sift, des_sift = sift.detectAndCompute(img, None)
    
    # ORB
    kp_orb, des_orb = orb.detectAndCompute(img, None)
    
    return kp_sift, des_sift, kp_orb, des_orb


target_kp_sift, target_des_sift, target_kp_orb, target_des_orb = detect_features(final_target)

target_desc = np.float32(target_des_sift)

# all_keypoints = kp_sift + kp_orb
    
# img_with_keypoints = cv2.drawKeypoints(edges, all_keypoints, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# cv2.imshow('img', img_with_keypoints)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [4]:
#Feature Matching

#FLANN
flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5),dict(checks=50))

#BF
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)


In [5]:
def match_fruit(descriptors):
    best_match = 0
    best_match_data = None

    for folder_path in os.listdir('Training/'):
        for img_path in os.listdir('Training/'+folder_path):
            img = cv2.imread('Training/'+folder_path+'/'+img_path)

            if img is not None:
                final_img = preprocess_img(img)

                kp_sift, des_sift, kp_orb, des_orb = detect_features(final_img)
                des_sift = np.float32(des_sift)
                des_orb = np.float32(des_orb)

                matches = flann.knnMatch(target_desc,des_sift,k=2)

                # matches_mask = [[0,0] for _ in range(len(matches))]
                
                good_matches = []

                curr_match = 0
                for i, (fn,sm) in enumerate(matches):
                    if fn.distance < 0.75 * sm.distance:
                        # matches_mask[i] = [1,0]
                        # curr_match+=1
                        good_matches.append(fn)
                
                if (len(good_matches)>4):
                    src = np.float32([target_kp_sift[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    dst = np.float32([kp_sift[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    
                    M, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

                    matches_mask = mask.ravel().tolist()
                    curr_match = sum(matches_mask)
                    if best_match < curr_match:
                        best_match = curr_match
                        best_match_data={
                            'img': img,
                            'kp': kp_sift,
                            'desc': des_sift,
                            'matches': good_matches,
                            'matches_mask':matches_mask
                        }
    
    # result = cv2.drawMatchesKnn(
    # target_img,
    # target_kp_sift,
    # best_match_data['img'],
    # best_match_data['kp'],
    # best_match_data['matches'],
    # None,
    # matchColor=[255,0,0],
    # matchesMask=best_match_data['matches_mask'],
    # singlePointColor=[0,0,255]
    # )

    result = cv2.drawMatches(
        target_img,
        target_kp_sift,
        best_match_data['img'],
        best_match_data['kp'],
        best_match_data['matches'],
        None,
        # matchColor=[255, 0, 0],
        matchesMask=best_match_data['matches_mask'],
        # singlePointColor=[0, 0, 255]
    )

    cv2.imshow('result',result)
    cv2.waitKey(0)

    return "Fruit Detected" if descriptors is not None else "No Fruit Detected"

# match_fruit(target_desc)

In [6]:
def making_model():
    desc = {}
    for fruit in os.listdir('Training/'):
        folder_path = os.path.join('Training', fruit)
        
        if os.path.isdir(folder_path):
            class_desc = []
            
            for img_path in os.listdir(folder_path):
                img = cv2.imread(os.path.join(folder_path, img_path))
                
                if img is not None:
                    final_img = preprocess_img(img)
                    kp_sift, des_sift, kp_orb, des_orb = detect_features(final_img)
                    
                    if des_sift is not None:
                        class_desc.append(np.float32(des_sift))
            
            if class_desc:
                desc[fruit] = np.vstack(class_desc)
    
    with open('fruit_detection_model.pkl', 'wb') as file:
        pickle.dump(desc, file)

# making_model()

In [7]:
descriptor_path = 'fruit_detection_model.pkl'

# with open(descriptor_path, 'rb') as file:
#     fruit_descriptors = pickle.load(file)

In [8]:
def match_fruit_model(descriptors):
    best_match = 0
    best_fruit = "Unknown"
    
    # Compare with each fruit's precomputed descriptors
    for fruit_class, stored_descriptors in fruit_descriptors.items():
        if descriptors is not None:
            matches = flann.knnMatch(descriptors, stored_descriptors, k=2)
            
            good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
            
            if len(good_matches) > best_match:
                best_match = len(good_matches)
                best_fruit = fruit_class
    
    return best_fruit if best_match > 0 else "No Fruit Detected"

In [ ]:
cap = cv2.VideoCapture(0)
def img_detection_live():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess image
        processed_frame = preprocess_img(frame)

        # Feature extraction memakai sift dan orb
        kp_sift, des_sift, kp_orb, des_orb = detect_features(processed_frame)

        frame_kp = cv2.drawKeypoints(frame, kp_sift, None, color=(0,255,0),
                                                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        # Image matching
        fruit_result = match_fruit_model(des_sift)

        # Display frame
        cv2.putText(frame_kp, fruit_result, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("Fruit Detection", frame_kp)

        # Kalau mw keluar pencet q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# img_detection_live()

NameError: name 'fruit_descriptors' is not defined